In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats


In [2]:
df1 = pd.read_csv('../../data/sweep_0_10_data.csv')
df2 = pd.read_csv('../../data/sweep_0_10_data2.csv')
df = pd.concat([df1, df2], axis=1)
df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp"], inplace=True)

EmptyDataError: No columns to parse from file

In [59]:
df.tail()

,amd,rcm,padding,matrix_name,partial_gauss,n,rank,max_mode_size,z,tile_size
19959,True,False,0,ex3,0,1821.0,9.0,607.0,52685.0,607.0
19960,True,False,0,ex3,0,1821.0,1.0,1821.0,52685.0,1821.0
19961,True,True,0,ex3,0,1821.0,38697.0,607.0,52685.0,3.0
19962,True,True,0,ex3,0,1821.0,9.0,607.0,52685.0,607.0
19963,True,True,0,ex3,0,1821.0,1.0,1821.0,52685.0,1821.0


In [81]:
df["log_obj_func"] = np.log(df["max_mode_size"]**6 + df["max_mode_size"]**3 *  df["rank"] + df["max_mode_size"]**2 *  df["rank"]**2)
df["obj_func"] = np.exp(df["log_obj_func"])

In [82]:
baseline_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
padding_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0)]
amd_df = df[(df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
rcm_df = df[(df['amd'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
partial_gauss_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['padding'] == 0)]

In [83]:
df.head()

,amd,rcm,padding,matrix_name,partial_gauss,n,rank,max_mode_size,z,tile_size,log_obj_func,obj_func
0,False,False,10,ex15,10,6867.0,33252.0,109.0,98623.0,3.0,30.329488,1.485693e+13
1,False,False,10,ex15,10,6867.0,8766.0,109.0,98623.0,7.0,28.587079,2.601421e+12
2,False,False,10,ex15,10,6867.0,6816.0,109.0,98623.0,9.0,28.436556,2.237893e+12
3,False,False,10,ex15,10,6867.0,2921.0,109.0,98623.0,21.0,28.208900,1.782254e+12
4,False,False,10,ex15,10,6867.0,539.0,109.0,98623.0,63.0,28.150559,1.681250e+12


In [84]:
# Distribution of log(I^6r^2)
fig = px.line(baseline_df, x="tile_size", y="log_obj_func", color='matrix_name', symbol="matrix_name", markers=True, log_x=True,
              labels={
                     "tile_size": "Tile size",
                     "matrix_name": "Matrix name",
                 }
              )
fig.update_layout(
    title={
        'text': "Influence of tile size choice on TT-MALS runtime",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()
fig.write_image("plots/baseline_tile_size_vs_log_obj_func.pdf")

In [85]:
fig = px.line(baseline_df, x="rank", y="max_mode_size", color="matrix_name", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank": "Maximum Rank (r)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size": "Maximum Mode Size (I)",
                 })
fig.update_layout(
    title={
        'text': "Trade-off between maximum rank and maximum mode size",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black') # Font color
)
fig.show()

fig.write_image("plots/baseline_max_mode_size_vs_max_rank.pdf")

In [118]:
# next: explore effects of variable ordering: rcm to concentrate nonzero entries
default_colorscale = px.colors.sequential.Jet
colors = px.colors.sample_colorscale(default_colorscale, 11)
fig = px.line(padding_df[padding_df["matrix_name"] == "ex3"], x="tile_size", y="log_obj_func", color="padding", symbol="padding", log_x=True, color_discrete_sequence=colors,
                 labels={
                     "tile_size": "Tile size",
                     "rcm": "RCM",
                     "matrix_name": "Matrix Name",
                 })
fig.update_layout(
    title={
        'text': "Influence of tile size choice and RCM on TT-MALS runtime",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()

fig.write_image("plots/rcm_tile_size_vs_log_obj_func.pdf")

In [100]:
fig = px.line(padding_df[padding_df["matrix_name"] == "ex10"], x="rank", y="max_mode_size", color="padding", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank": "Maximum Rank (r)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size": "Maximum Mode Size (I)",
                 })
fig.update_layout(
    title={
        'text': "Influence of tile size choice and RCM on TT-MALS runtime",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()

fig.write_image("plots/rcm_max_mode_size_vs_max_rank.pdf")

In [95]:
rcm_df

,amd,rcm,padding,matrix_name,partial_gauss,n,rank,max_mode_size,z,tile_size,log_obj_func,obj_func
2184,False,False,0,ex15,0,6867.0,19825.0,109.0,98671.0,3.0,29.482993,6.372371e+12
2185,False,False,0,ex15,0,6867.0,8125.0,109.0,98671.0,7.0,28.536030,2.471954e+12
2186,False,False,0,ex15,0,6867.0,6175.0,109.0,98671.0,9.0,28.390951,2.138127e+12
2187,False,False,0,ex15,0,6867.0,2275.0,109.0,98671.0,21.0,28.185790,1.741538e+12
2188,False,False,0,ex15,0,6867.0,325.0,109.0,98671.0,63.0,28.149086,1.678776e+12
...,...,...,...,...,...,...,...,...,...,...,...,...
9977,False,False,0,ex3,0,1821.0,7.0,607.0,52685.0,607.0,38.451173,5.001867e+16
9978,False,False,0,ex3,0,1821.0,1.0,1821.0,52685.0,1821.0,45.042846,3.646361e+19
9979,False,True,0,ex3,0,1821.0,20785.0,607.0,52685.0,3.0,38.454443,5.018250e+16
9980,False,True,0,ex3,0,1821.0,7.0,607.0,52685.0,607.0,38.451173,5.001867e+16
